In [ ]:
import datetime as dt
import utils
from sentinelsat import SentinelAPI
import json
import rasterio
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from rasterio import plot

In [ ]:
geojson_path = "data/var.geojson"
credentials_path = "secrets/sentinel_api_credentials.json"
wildfire_date = dt.date(2021, 8, 17)
observation_interval = 15
path = 'data/'
output_folder = 'output/'

In [ ]:
with open(credentials_path, 'r') as infile:
    credentials = json.load(infile)

api = SentinelAPI(
    credentials["username"],
    credentials["password"]
)

In [ ]:
before_image, after_image = utils.get_before_after_images(
    api=api, wildfire_date=wildfire_date, geojson_path=geojson_path,
    observation_interval=observation_interval, path=path,
    band1='B04', band2='B08', output_folder=output_folder,
    resolution=10, cloud_threshold=40
)

In [ ]:
# plot before_image and after_image side by side
fig, ax = plt.subplots(1, 2, figsize=(12, 12))
ax[0].imshow(before_image)
ax[1].imshow(after_image)
plt.show()

In [ ]:
diff = before_image - after_image

plt.figure(figsize=(10, 10))
plt.imshow(diff)
plt.show()

In [ ]:
thresholded = utils.threshold_filter(diff, threshold=0.25)

plt.figure(figsize=(10, 10))
plt.imshow(diff)
plt.show()

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(12, 12))
axs[0, 0].imshow(before_image)
axs[0, 1].imshow(after_image)
axs[1, 0].imshow(diff)
axs[1, 1].imshow(thresholded)
plt.show()

---
Running the following cells is equivalent to calling `get_before_after_images` directly

In [ ]:
dates = (dt.date(2021, 8, 2), dt.date(2021, 8, 16))
images_df = utils.get_dataframe_between_dates(api, dates[0], dates[1], geojson_path)

In [ ]:
images_df = utils.minimize_dataframe(images_df)

In [ ]:
key_cols = ["cloudcoverpercentage", "vegetationpercentage",
            "waterpercentage", "size", "ingestiondate", "score"]
images_df[key_cols]

In [ ]:
uuid, title = utils.get_uuid_title(images_df)

In [ ]:
path = './data/'
utils.download_from_api(api, uuid, title, path)

In [ ]:
corr_matrix = images_df.corr()

plt.figure(figsize=(10, 10))
hm = sns.heatmap(corr_matrix, linewidths=1, linecolor='gray', cmap='coolwarm')
hm.set_yticklabels(hm.get_yticklabels(), fontsize=11)
hm.set_xticklabels(hm.get_xticklabels(), rotation=45, fontsize=11, ha='right')
plt.show()

In [ ]:
wildfire_date = dt.date(2021, 8, 17)
observation_interval = 15

In [ ]:
test1 = utils.get_image(api=api, wildfire_date=wildfire_date,
    observation_interval=observation_interval, when='before',
    geojson_path=geojson_path, path='data/',
    band1='B04', band2='B08', output_folder='output/', resolution=10)

In [ ]:
plt.figure(figsize=(16, 9))
plot.show(test1)
plt.show()

In [ ]:
test2 = utils.get_image(api=api, wildfire_date=wildfire_date,
    observation_interval=observation_interval, when='after',
    geojson_path=geojson_path, path='data/',
    band1='B04', band2='B08', output_folder='output/', resolution=10)

In [ ]:
plt.figure(figsize=(16, 9))
plot.show(test2)
plt.show()

In [ ]:
diff = test1 - test2

plt.figure(figsize=(16, 9))
plt.imshow(diff)
plt.show()

In [ ]:
diff = utils.threshold_filter(diff, 0.25)

plt.figure(figsize=(16, 9))
plt.imshow(diff)
plt.show()